In [1]:
import abem as ab
import numpy as np
from numpy.linalg import norm


# Test Problem 1
# Dirichlet boundary condition with phi chose to be the same
# as a point-source at the center of the square.
#
frequency = 100.0 # frequency [Hz]
k = ab.frequency_to_wavenumber(frequency)

solver = ab.ExteriorHelmholtzSolver3D(ab.sphere())

boundary_condition = solver.dirichlet_boundary_condition()
for i in range(solver.centers.shape[0]):
    R = norm(solver.centers[i, :])
    boundary_condition.f[i] = np.exp(1j * k * R) / R

boundary_incidence = ab.BoundaryIncidence(solver.len())
boundary_incidence.phi.fill(0.0)
boundary_incidence.v.fill(0.0)

exterior_points = np.array([[0.0000, 0.0000,  2.0000],
                           [0.0500, 0.0000,  4.0000],
                           [0.1000, 0.0000,  8.0000],
                           [0.0500, 0.1000, -2.0000]], dtype=np.float32)

exterior_incident_phi = np.zeros(exterior_points.shape[0], dtype=np.complex64)

boundary_solution = solver.solve_boundary(k, boundary_condition, boundary_incidence)
sample_solution = boundary_solution.solve_samples(exterior_incident_phi, exterior_points)
print("\n\nTest Problem 1")
print("==============\n")
print(boundary_solution)
print(sample_solution)


# Test Problem 2
# Neumann boundary condition with v chosen to be the same
# as a point-source at the center of the square.
#
boundary_condition = solver.neumann_boundary_condition()
centers = solver.geometry.centers()
normals = solver.geometry.normals()
for i in range(solver.centers.shape[0]):
    R = norm(centers[i, :])
    dot = np.dot(solver.centers[i, :], normals[i])
    boundary_condition.f[i] = np.exp(1j * k * R) * dot * (1j * k * R - 1.0) / R**3

boundary_solution = solver.solve_boundary(k, boundary_condition, boundary_incidence)
sample_solution = boundary_solution.solve_samples(exterior_incident_phi, exterior_points)
print("\n\nTest Problem 2")
print("==============\n")
print(boundary_solution)
print(sample_solution)


# Test Problem 3
# Dirichlet boundary condition on the full boundary.
# The 400Hz field in this case is generated by a point source
# at (0.5, 0.25)
boundary_condition = solver.dirichlet_boundary_condition()
source_point = np.array([0.0, 0.0, 3.0], dtype=np.float32)
for i in range(solver.centers.shape[0]):
    r = source_point - solver.centers[i, :]
    R = norm(r)
    boundary_incidence.phi[i] = np.exp(1j * k * R) / (4.0 * np.pi * R)
    boundary_condition.f[i] = boundary_incidence.phi[i]
    dot = -np.dot(r, normals[i])
    boundary_incidence.v[i] = np.exp(1j * k * R) * dot * (1j * k * R - 1.0) / (4.0 * np.pi * R**3)

boundary_solution = solver.solve_boundary(k, boundary_condition, boundary_incidence)

for i in range(exterior_points.shape[0]):
    R = norm(exterior_points[i, :] - source_point)
    exterior_incident_phi[i] = np.exp(1j * k * R) / (4.0 * np.pi * R)
    
sample_solution = boundary_solution.solve_samples(exterior_incident_phi, exterior_points)
print("\n\nTest Problem 3")
print("==============\n")
print(boundary_solution)
print(sample_solution)


# Test Problem 4
# Neumann boundary condition with v=0 on the full boundary.
# The 400Hz field in this case is generated by a point source
# at (0.5, 0.25)
boundary_condition = solver.neumann_boundary_condition()

source_point = np.array([0.0, 0.0, 3.0], dtype=np.float32)
for i in range(solver.centers.shape[0]):
    r = source_point - solver.centers[i, :]
    R = norm(r)
    dot = -np.dot(r, normals[i])
    boundary_incidence.v[i] = np.exp(1j * k * R) * dot * (1j * k * R - 1.0) / (4.0 * np.pi * R**3)

boundary_solution = solver.solve_boundary(k, boundary_condition, boundary_incidence)

for i in range(exterior_points.shape[0]):
    R = norm(exterior_points[i, :] - source_point)
    exterior_incident_phi[i] = np.exp(1j * k * R) / (4.0 * np.pi * R)
    
sample_solution = boundary_solution.solve_samples(exterior_incident_phi, exterior_points)
print("\n\nTest Problem 4")
print("==============\n")
print(boundary_solution)
print(sample_solution)




Test Problem 1

Density of medium:      1.205 kg/m^3
Speed of sound:         344.0 m/s
Wavenumber (Frequency): 1.826507356738252 (100.0 Hz)

index   Potential               Pressure                 Velocity                 Intensity

    1  -5.9354e-02+1.1234e+00  -8.5057e+02-4.4938e+01i  -1.8609e+00-1.1710e+00i   8.1774e+02
    2  -5.9402e-02+1.1234e+00  -8.5054e+02-4.4974e+01i  -1.8610e+00-1.1710e+00i   8.1777e+02
    3  -5.9354e-02+1.1234e+00  -8.5057e+02-4.4938e+01i  -1.8609e+00-1.1711e+00i   8.1774e+02
    4  -5.9354e-02+1.1234e+00  -8.5057e+02-4.4938e+01i  -1.8609e+00-1.1710e+00i   8.1774e+02
    5  -5.9402e-02+1.1234e+00  -8.5054e+02-4.4974e+01i  -1.8610e+00-1.1710e+00i   8.1777e+02
    6  -5.9354e-02+1.1234e+00  -8.5057e+02-4.4938e+01i  -1.8609e+00-1.1711e+00i   8.1774e+02
    7  -4.8280e-02+1.1310e+00  -8.5632e+02-3.6554e+01i  -1.8703e+00-1.1178e+00i   8.2122e+02
    8   1.0774e-02+1.1696e+00  -8.8553e+02+8.1574e+00i  -1.9635e+00-1.1314e+00i   8.6474e+02
    9  -4.8379e-02+1